# Tratamento e Limpeza de Dados

Este notebook realiza o tratamento e a padronização dos dados da camada **Bronze**, preparando-os para a camada **Silver** dentro da arquitetura *medallion*.

O objetivo é transformar dados brutos em dados **limpos, consistentes e prontos para análise**.


In [23]:
# Bibliotecas necessárias
import pandas as pd
import os
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

In [24]:
# Leitura do CSV original
csv_path = os.path.join('Data Layer', 'raw', 'ncr_ride_bookings.csv')

# Tentar diferentes caminhos possíveis
if not os.path.exists(csv_path):
    # Caminho alternativo (Kaggle)
    csv_path = "/kaggle/input/uber-analise/ncr_ride_bookings.csv"
    if not os.path.exists(csv_path):
        # Caminho relativo da pasta Transformer
        csv_path = os.path.join('..', 'Data Layer', 'raw', 'ncr_ride_bookings.csv')

print(f"Carregando dados de: {csv_path}")
df_bronze = pd.read_csv(csv_path)

print(f"Total de registros carregados: {len(df_bronze):,}")
df_bronze.head()

Carregando dados de: ..\Data Layer\raw\ncr_ride_bookings.csv
Total de registros carregados: 150,000
Total de registros carregados: 150,000


,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,...,Reason for cancelling by Customer,Cancelled Rides by Driver,Driver Cancellation Reason,Incomplete Rides,Incomplete Rides Reason,Booking Value,Ride Distance,Driver Ratings,Customer Rating,Payment Method
0,2024-03-23,12:29:38,"""CNR5884300""",No Driver Found,"""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-29,18:01:39,"""CNR1326809""",Incomplete,"""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,...,NaN,NaN,NaN,1.0,Vehicle Breakdown,237.0,5.73,NaN,NaN,UPI
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,...,NaN,NaN,NaN,NaN,NaN,627.0,13.58,4.9,4.9,Debit Card
3,2024-10-21,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,...,NaN,NaN,NaN,NaN,NaN,416.0,34.02,4.6,5.0,UPI
4,2024-09-16,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,...,NaN,NaN,NaN,NaN,NaN,737.0,48.21,4.1,4.3,UPI


## Nome das Colunas

Padronização dos nomes das colunas para garantir consistência e compatibilidade com o banco de dados.  
Essa etapa ajusta os nomes para seguir o padrão definido no data warehouse (uso de snake_case, remoção de espaços e caracteres especiais).

In [25]:
# Cria uma cópia da camada Bronze
df_silver = df_bronze.copy()

# Renomeia colunas para o padrão unificado (Silver)
df_silver.rename(columns={
    'Booking ID': 'Booking_ID',
    'Booking Status': 'Booking_Status',
    'Customer ID': 'Customer_ID',
    'Vehicle Type': 'Vehicle_Type',
    'Pickup Location': 'Pickup_Location',
    'Drop Location': 'Drop_Location',
    'Avg VTAT': 'Avg_VTAT',
    'Avg CTAT': 'Avg_CTAT',
    'Reason for cancelling by Customer': 'Reason_for_cancelling_by_Customer',
    'Driver Cancellation Reason': 'Driver_Cancellation_Reason',
    'Incomplete Rides Reason': 'Incomplete_Rides_Reason',
    'Booking Value': 'Booking_Value',
    'Ride Distance': 'Ride_Distance',
    'Driver Ratings': 'Driver_Ratings',
    'Customer Rating': 'Customer_Rating',
    'Payment Method': 'Payment_Method'
}, inplace=True)

df_silver.head()


,Date,Time,Booking_ID,Booking_Status,Customer_ID,Vehicle_Type,Pickup_Location,Drop_Location,Avg_VTAT,Avg_CTAT,...,Reason_for_cancelling_by_Customer,Cancelled Rides by Driver,Driver_Cancellation_Reason,Incomplete Rides,Incomplete_Rides_Reason,Booking_Value,Ride_Distance,Driver_Ratings,Customer_Rating,Payment_Method
0,2024-03-23,12:29:38,"""CNR5884300""",No Driver Found,"""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-29,18:01:39,"""CNR1326809""",Incomplete,"""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,...,NaN,NaN,NaN,1.0,Vehicle Breakdown,237.0,5.73,NaN,NaN,UPI
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,...,NaN,NaN,NaN,NaN,NaN,627.0,13.58,4.9,4.9,Debit Card
3,2024-10-21,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,...,NaN,NaN,NaN,NaN,NaN,416.0,34.02,4.6,5.0,UPI
4,2024-09-16,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,...,NaN,NaN,NaN,NaN,NaN,737.0,48.21,4.1,4.3,UPI


## Remoção de Coluna Redundante

As colunas de flags apresentam informações que já estão representadas na variável Booking_status. Portanto, mantê-las seria redundante e poderia causar duplicidade semântica ou inconsistências nas análises.

In [26]:
# Remove colunas de flags irrelevantes
df_silver.drop(columns=[
    'Cancelled Rides by Customer', 
    'Cancelled Rides by Driver', 
    'Incomplete Rides'
], inplace=True, errors='ignore')

## Tratamento de Texto

Aplicando tratamento de texto nas colunas necessárias.

In [27]:
# Remove aspas desnecessárias
df_silver['Booking_ID'] = df_silver['Booking_ID'].astype(str).str.replace('"', '', regex=False)
df_silver['Customer_ID'] = df_silver['Customer_ID'].astype(str).str.replace('"', '', regex=False)

## Remoção de Tuplas com o Identificador Único Duplicado

Existe um erro no dataset que permite que algumas tuplas tenham o Booking_ID duplicado

In [28]:
# Remove dados que possuem a coluna 'Booking_ID' duplicada
initial_count = len(df_silver)
df_silver.drop_duplicates(subset=['Booking_ID'], keep='first', inplace=True)
final_count = len(df_silver)
print(f"Quantidade de dados duplicados: {initial_count - final_count}")

Quantidade de dados duplicados: 1233


## Converte a coluna de data e hora

In [ ]:
# Converte Date (formato padrão YYYY-MM-DD)
df_silver['Date'] = pd.to_datetime(df_silver['Date'], errors='coerce').dt.date

# Converte Time - MANTER COMO STRING para garantir inserção correta no PostgreSQL
# Não converter para time object do Python, pois causa problemas no psycopg2
df_silver['Time'] = pd.to_datetime(df_silver['Time'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M:%S')

## Remoção de outliers usando método do IQR (Interquartile Range)

Esta etapa identifica e remove valores extremos nas colunas numéricas do dataset Silver para garantir consistência estatística:

Colunas analisadas: Avg_VTAT, Avg_CTAT, Booking_Value, Ride_Distance

Método utilizado: Intervalo Interquartil (IQR)

Calcula Q1 (25%) e Q3 (75%) para cada coluna.

Determina o IQR = Q3 - Q1.

Define limites aceitáveis: [Q1 - 1.5*IQR, Q3 + 1.5*IQR].

Remove linhas com valores fora desses limites.

In [30]:
# Lista de colunas numéricas para análise
numeric_cols = ['Avg_VTAT', 'Avg_CTAT', 'Booking_Value', 'Ride_Distance']

def remove_outliers_iqr(df, cols):
    cleaned_df = df.copy()
    outlier_counts = {}
    
    for col in cols:
        if col not in df.columns:
            continue
        
        # Calcula Q1, Q3 e IQR
        Q1 = cleaned_df[col].quantile(0.25)
        Q3 = cleaned_df[col].quantile(0.75)
        IQR = Q3 - Q1

        # Define limites inferior e superior
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Conta quantos seriam removidos
        initial_len = len(cleaned_df)
        cleaned_df = cleaned_df[(cleaned_df[col] >= lower_bound) & (cleaned_df[col] <= upper_bound)]
        final_len = len(cleaned_df)
        outlier_counts[col] = initial_len - final_len

    return cleaned_df, outlier_counts


# Aplica a função
df_silver_clean, outlier_summary = remove_outliers_iqr(df_silver, numeric_cols)

print("Outliers removidos com base no método IQR:")
for col, count in outlier_summary.items():
    print(f"  • {col}: {count} linhas removidas")

print(f"\nLinhas finais após limpeza: {len(df_silver_clean)} (de {len(df_silver)})")

# Substitui o DataFrame antigo pelo limpo
df_silver = df_silver_clean


Outliers removidos com base no método IQR:
  • Avg_VTAT: 10401 linhas removidas
  • Avg_CTAT: 37191 linhas removidas
  • Booking_Value: 3410 linhas removidas
  • Ride_Distance: 0 linhas removidas

Linhas finais após limpeza: 97765 (de 148767)


## Tratamento de valores ausentes (NaN)

Esta etapa realiza o tratamento de dados nulos para garantir consistência e integridade do dataset Silver:

**Motivos de cancelamento:** substitui valores ausentes por "Reason Unknown".

**Colunas numéricas:** converte para tipo numérico seguro e substitui valores nulos pela média da coluna.

**Método de pagamento:** preenche valores ausentes com a moda (valor mais frequente).

In [31]:
# Substitui valores ausentes por texto padrão nas colunas de motivos
for col in ['Incomplete_Rides_Reason', 'Driver_Cancellation_Reason', 'Reason_for_cancelling_by_Customer']:
    if col in df_silver.columns:
        df_silver[col] = df_silver[col].fillna('Reason Unknown')

# Converte colunas numéricas para tipo numérico seguro
num_cols = ['Avg_VTAT', 'Avg_CTAT', 'Booking_Value', 'Ride_Distance', 'Driver_Ratings', 'Customer_Rating']
for col in num_cols:
    if col in df_silver.columns:
        df_silver[col] = pd.to_numeric(df_silver[col], errors='coerce')

# Imputa a média nas colunas numéricas
for col in num_cols:
    if col in df_silver.columns:
        df_silver[col] = df_silver[col].fillna(df_silver[col].mean())

# Imputa a moda na coluna de método de pagamento (se existir)
if 'Payment_Method' in df_silver.columns:
    mode_value = df_silver['Payment_Method'].mode(dropna=True)
    if not mode_value.empty:
        df_silver['Payment_Method'] = df_silver['Payment_Method'].fillna(mode_value[0])

In [32]:
# Visualiza uma amostra dos dados transformados
df_silver.head(10)

,Date,Time,Booking_ID,Booking_Status,Customer_ID,Vehicle_Type,Pickup_Location,Drop_Location,Avg_VTAT,Avg_CTAT,Reason_for_cancelling_by_Customer,Driver_Cancellation_Reason,Incomplete_Rides_Reason,Booking_Value,Ride_Distance,Driver_Ratings,Customer_Rating,Payment_Method
1,2024-11-29,18:01:39,CNR1326809,Incomplete,CID4604802,Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,Reason Unknown,Reason Unknown,Vehicle Breakdown,237.0,5.73,4.230757,4.403841,UPI
2,2024-08-23,08:56:10,CNR8494506,Completed,CID9202816,Auto,Khandsa,Malviya Nagar,13.4,25.8,Reason Unknown,Reason Unknown,Reason Unknown,627.0,13.58,4.900000,4.900000,Debit Card
3,2024-10-21,17:17:25,CNR8906825,Completed,CID2610914,Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,Reason Unknown,Reason Unknown,Reason Unknown,416.0,34.02,4.600000,5.000000,UPI
4,2024-09-16,22:08:00,CNR1950162,Completed,CID9933542,Bike,Ghitorni Village,Khan Market,5.3,19.6,Reason Unknown,Reason Unknown,Reason Unknown,737.0,48.21,4.100000,4.300000,UPI
5,2024-02-06,09:44:56,CNR4096693,Completed,CID4670564,Auto,AIIMS,Narsinghpur,5.1,18.1,Reason Unknown,Reason Unknown,Reason Unknown,316.0,4.85,4.100000,4.600000,UPI
6,2024-06-17,15:45:58,CNR2002539,Completed,CID6800553,Go Mini,Vaishali,Punjabi Bagh,7.1,20.4,Reason Unknown,Reason Unknown,Reason Unknown,640.0,41.24,4.000000,4.100000,UPI
7,2024-03-19,17:37:37,CNR6568000,Completed,CID8610436,Auto,Mayur Vihar,Cyber Hub,12.1,16.5,Reason Unknown,Reason Unknown,Reason Unknown,136.0,6.56,4.400000,4.200000,UPI
9,2024-12-16,19:06:48,CNR7721892,Incomplete,CID5214275,Auto,Rohini,Adarsh Nagar,6.1,26.0,Reason Unknown,Reason Unknown,Other Issue,135.0,10.36,4.230757,4.403841,Cash
10,2024-06-14,16:24:12,CNR9070334,Completed,CID6680340,Auto,Udyog Bhawan,Dwarka Sector 21,7.7,18.9,Reason Unknown,Reason Unknown,Reason Unknown,181.0,19.84,4.200000,4.900000,Cash
13,2024-09-11,19:29:39,CNR2987763,Completed,CID2669710,Go Mini,Malviya Nagar,Ghitorni Village,12.2,28.2,Reason Unknown,Reason Unknown,Reason Unknown,394.0,21.44,4.100000,4.700000,UPI


## Exportação dos Dados Tratados

In [ ]:
import psycopg2
from psycopg2 import extras
from dotenv import load_dotenv

# Verificar se df_silver está definido
if 'df_silver' not in locals():
    raise NameError("df_silver não está definido. Execute todas as células anteriores primeiro.")

print(f"Total de registros a serem salvos: {len(df_silver):,}")

# Caminho para salvar a camada Silver
SILVER_PATH = os.path.join('Data Layer', 'silver', 'uber_silver.csv')
os.makedirs(os.path.dirname(SILVER_PATH), exist_ok=True)

df_silver.to_csv(SILVER_PATH, index=False)
print(f"Dados transformados salvos em '{SILVER_PATH}'.")

# Função para obter string de conexão (usa .env se disponível)
def get_db_connection_info():
    load_dotenv()
    url = os.getenv('DB_URL')
    db_env = os.getenv('DB_ENV')
    if url is not None and db_env == 'prod':
        return url

    DB_USER = os.getenv('DB_USER', 'postgres')
    DB_PASSWORD = os.getenv('DB_PASSWORD', 'postgres')
    DB_HOST = os.getenv('DB_HOST', 'localhost')
    DB_PORT = os.getenv('DB_PORT', '5432')
    DB_NAME = os.getenv('DB_NAME', 'uber')

    return {
        'host': DB_HOST,
        'dbname': DB_NAME,
        'user': DB_USER,
        'password': DB_PASSWORD,
        'port': DB_PORT
    }

# Carregar DDL e inserir
# Tentar múltiplos caminhos para o arquivo DDL
DDL_FILE = os.path.join('Data Layer', 'silver', 'silver_ddl.sql')
if not os.path.exists(DDL_FILE):
    DDL_FILE = os.path.join('..', 'Data Layer', 'silver', 'silver_ddl.sql')
if not os.path.exists(DDL_FILE):
    print(f"AVISO: Arquivo DDL não encontrado em: {DDL_FILE}")
    print("Tentando conectar sem executar DDL...")

conn_info = get_db_connection_info()

try:
    print('\nIniciando carregamento na camada SILVER (Postgres)...')
    df_to_load = df_silver.copy()

    # Ajustes de tipos: garantir colunas coerentes
    if 'Date' in df_to_load.columns:
        df_to_load['Date'] = pd.to_datetime(df_to_load['Date'], errors='coerce').dt.date
    if 'Time' in df_to_load.columns:
        # Converter Time para string formato HH:MM:SS, mantendo compatibilidade com PostgreSQL
        # Primeiro converte para datetime, depois para string
        time_col = pd.to_datetime(df_to_load['Time'], errors='coerce', format='%H:%M:%S')
        # Aplicar strftime apenas nos valores não-nulos
        df_to_load['Time'] = time_col.apply(lambda x: x.strftime('%H:%M:%S') if pd.notna(x) else None)

    # Conectar ao banco
    conn = psycopg2.connect(**conn_info)
    cur = conn.cursor()
    
    # Lê e executa DDL se o arquivo existir
    if os.path.exists(DDL_FILE):
        print(f'Executando DDL de: {DDL_FILE}')
        with open(DDL_FILE, 'r', encoding='utf-8') as f:
            ddl_sql = f.read()
        cur.execute(ddl_sql)
        conn.commit()
    else:
        print('Arquivo DDL não encontrado. Assumindo que schema e tabela já existem.')
    
    # Limpar tabela antes de inserir
    print('Limpando tabela anterior...')
    try:
        cur.execute('TRUNCATE TABLE silver.uber_silver')
        conn.commit()
    except Exception as truncate_error:
        print(f'Aviso ao limpar tabela: {truncate_error}')
        print('Tentando criar tabela se não existir...')
        # Se a tabela não existe, tentar criar com DDL básico
        if not os.path.exists(DDL_FILE):
            print('Criando schema e tabela...')
            cur.execute('CREATE SCHEMA IF NOT EXISTS silver')
            conn.commit()


    # Inserção em batches
    table = 'silver.uber_silver'
    cols = list(df_to_load.columns)
    # Map column names to DDL names (simple mapping)
    cols = [c for c in cols if c.lower() != 'index']
    insert_query = f"INSERT INTO {table} ({', '.join(cols)}) VALUES ({', '.join(['%s'] * len(cols))})"

    records = [tuple(None if (pd.isna(v)) else v for v in row) for row in df_to_load[cols].itertuples(index=False, name=None)]
    extras.execute_batch(cur, insert_query, records, page_size=1000)
    conn.commit()

    print(f'Carga concluída: {len(records)} linhas inseridas em {table}.')

except Exception as e:
    print('Erro durante carregamento na camada SILVER:', e)
    print('\nVerifique:')
    print('1. Se o PostgreSQL está rodando')
    print('2. Se o arquivo .env está configurado corretamente')
    print('3. Se o schema e tabela existem no banco')
    if 'conn' in locals():
        conn.rollback()
finally:
    if 'cur' in locals():
        cur.close()
    if 'conn' in locals():
        conn.close()

print('\nProcesso ETL Raw -> Silver concluído!')

Total de registros a serem salvos: 97,765
Dados transformados salvos em 'Data Layer\silver\uber_silver.csv'.

Iniciando carregamento na camada SILVER (Postgres)...
Executando DDL de: ..\Data Layer\silver\silver_ddl.sql
Limpando tabela anterior...
Dados transformados salvos em 'Data Layer\silver\uber_silver.csv'.

Iniciando carregamento na camada SILVER (Postgres)...
Executando DDL de: ..\Data Layer\silver\silver_ddl.sql
Limpando tabela anterior...
Carga concluída: 97765 linhas inseridas em silver.uber_silver.

Processo ETL Raw -> Silver concluído!
Carga concluída: 97765 linhas inseridas em silver.uber_silver.

Processo ETL Raw -> Silver concluído!
